Name: Josemaria Macedo Carrillo\
Ttile: Label Matrix Data Exploration\
Created: 06/22/23\
Last modified:\
DSI

In [106]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [107]:
# Check what the datasets contain
contracts = pd.read_csv("../data/land matrix/contracts.csv", delimiter=";", parse_dates=["Contract date"])
print("Contracts data:\n", contracts.head())

# This dataset can be useful to check the data source of the deal
datasources = pd.read_csv("../data/land matrix/datasources.csv", delimiter=";")
print("Data sources:\n", datasources.head())

# This is the main data source but I don't know if we have to focus on the "Top parent companies" or the "Operating company: Investor ID"
deals = pd.read_csv("../data/land matrix/deals.csv", delimiter=";")
print("Deals variables:\n", list(deals.columns))

investors = pd.read_csv("../data/land matrix/investors.csv", delimiter=";")
print("Investors:\n", investors.head())

# Useful to see relation between companies and their parent companies
involvements = pd.read_csv("../data/land matrix/involvements.csv", delimiter=";")
print("Involvements:\n", involvements.head())

# Land regional location data. The data is at the regional level
locations = pd.read_csv("../data/land matrix/locations.csv", delimiter=";")
print("Locations:\n", locations.head())

# How many regions are there? There should be 27 regions but there are 1272 unique values so I guess some regions are written differently or maybe they go down to municipalities (hromadas)
# Reference: https://en.wikipedia.org/wiki/Administrative_divisions_of_Ukraine
print("Number of administrative regions:", locations.loc[:, "Location"].nunique())
# locations.loc[:, "Location"].values

Contracts data:
          ID  Deal ID Contract number Contract date Contract expiration date  \
0  CPgZ6sbB        8             NaN           NaT                      NaN   
1  YHHYw__E       11             NaN    2005-04-26                      NaN   
2  kF4FrvdI       12             NaN    2008-12-08                      NaN   
3  iecRI3z0       13             NaN    2009-12-08                      NaN   
4  MCSdnge5       14             NaN    1996-01-09                      NaN   

   Duration of the agreement  \
0                       99.0   
1                       70.0   
2                       70.0   
3                       70.0   
4                        NaN   

                                 Comment on contract  
0              long-term land lease of upto 99 years  
1  On 12 May 2006, The Cambodian Ministry approve...  
2  Legal Papers and Rights for Investment Timelin...  
3                                                NaN  
4  Progress of Implementation after sign

/var/folders/j4/m2rbwhqs6yq26cq7rc0g4cm80000gn/T/ipykernel_2712/2085513018.py:10: DtypeWarning: Columns (139) have mixed types. Specify dtype option on import or set low_memory=False.
  deals = pd.read_csv("../data/land matrix/deals.csv", delimiter=";")


In [108]:
# Data time period from 2004 to 2022
contracts_c = contracts.copy()
contracts_c.loc[:, "year"] = contracts_c.loc[:, "Contract date"].dt.year

contracts_c.loc[:, "year"].unique()

array([  nan, 2005., 2008., 2009., 1996., 2006., 2001., 1999., 1998.,
       2000., 2007., 2004., 2002., 2022., 2011., 1994., 1995., 2012.,
       2010., 2003., 2013., 1976., 2018., 2038., 2017., 1986., 1993.,
       1990., 1991., 1985., 1959., 2019., 1997., 2014., 2015., 1989.,
       2021., 2016., 1983., 2020., 1966., 1960., 1977., 1992., 2029.,
       1987.])

In [109]:
# Check if Astarta has any land bought from 2004 to 2022
deals_c = deals.copy()
deals_c.loc[:, "name_low"] = deals_c.loc[:, "Operating company: Name"].str.lower()
deals_c.loc[:, "parent_low"] = deals_c.loc[:, "Top parent companies"].str.lower()

# There's no mention of Astarta as a parent company but there's one match
# for deal ID 1037 as an operating company
deals_c.loc[deals_c.loc[:, "parent_low"].str.contains("astarta", na = False)]
deals_c = deals_c.loc[deals_c.loc[:, "name_low"].str.contains("astarta")]

# For some reason there's no info on that deal in the contracts csv so we can
# get the date of the deal
deals_c.merge(contracts, on="Deal ID")

# Not even if we filter the contracts dataset with the corresponding deal id
contracts.loc[contracts.loc[:, "Deal ID"] == 1037]

,ID,Deal ID,Contract number,Contract date,Contract expiration date,Duration of the agreement,Comment on contract
